Total Game Score Model - Model Evaluation - GBM

In [ ]:
import pandas as pd
import numpy as np
import joblib
import warnings
warnings.filterwarnings('ignore')

import sys
sys.path.append('..')
from total_points_model.domain.preprocessing.data_preprocessor import DataPreprocessor
from total_points_model.domain.modelling.hyperparameter_tuning import XGBHyperparameterTuner
from total_points_model.domain.modelling.supermodel import SuperXGBRegressor
from total_points_model.domain.modelling.model_evaluation import XGBRegressorEvaluator

pd.options.display.max_rows = 100
pd.options.display.max_columns = 999

%load_ext autoreload
%autoreload 2

Load Data

In [ ]:
afl_data = pd.read_csv("data/merged-data/afl_match_data.csv")

In [ ]:
afl_data['ModellingFilter'] = np.where(afl_data['Date'] < "2019-01-01", True, False)
afl_data = afl_data[afl_data['Year'] >= 2010]

afl_data = afl_data.rename(columns = {'Total Game Score':'Total_Game_Score'})

In [ ]:
training_data = afl_data[afl_data['ModellingFilter']]
test_data = afl_data[~afl_data['ModellingFilter']]

In [ ]:
response = "Total_Game_Score"

In [ ]:
X_train, y_train = training_data.drop(columns = [response]), training_data[response]
X_test, y_test = test_data.drop(columns = [response]), test_data[response]

In [ ]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

Preprocess Data

In [ ]:
afl_rename_dict = {
    'Total Game Score':'Total_Game_Score',
    'Home Win':'Home_Win'
    }

In [ ]:
preprocessor = DataPreprocessor(rename_dict=afl_rename_dict)

In [ ]:
preprocessor.fit(X_train)

In [ ]:
X_train_preproc = preprocessor.transform(X_train)
X_test_preproc = preprocessor.transform(X_test)

In [ ]:
X_train_preproc.head()

In [ ]:
X_train_preproc.shape, X_test_preproc.shape

Load Model

In [ ]:
xgb_model = joblib.load("/models/xgb_regression_test_v1.joblib")

Get Predictions

In [ ]:
train_preds = xgb_model.predict(X_train_preproc)
test_preds = xgb_model.predict(X_test_preproc)

Evaluate Model

In [ ]:
test_evaluation_data = X_test_preproc.copy()
test_evaluation_data['xgb_preds'] = xgb_model.predict(X_test_preproc)
test_evaluation_data['old_preds'] = 170
test_evaluation_data[response] = test_data[response].reset_index(drop=True)
test_evaluation_data.head()

In [ ]:
test_xgb_evals = XGBRegressorEvaluator(model = xgb_model,
                                       data = test_evaluation_data,
                                       actual_name = response,
                                       expected_name = "xgb_preds",
                                       compare_name = "old_preds"
                                       )

In [ ]:
test_xgb_evals.plot_feature_importance(max_num_features=10, importance_type="total_gain")

In [ ]:
test_xgb_evals.plot_shap_summary_plot()

In [ ]:
feature_list = [
    'Home_Total_Q4_Score_avg2'
]

In [ ]:
test_xgb_evals.plot_pdp(feature_list)

In [ ]:
test_xgb_evals.plot_ice(feature_list)

In [ ]:
test_xgb_evals.plot_ave()

In [ ]:
test_xgb_evals.plot_feature_ave('Venue_Docklands')

In [ ]:
test_xgb_evals.plot_feature_ave('Home_Total_Q4_Score_avg2')

Model Metrics

In [ ]:
test_xgb_evals.get_mae()

In [ ]:
test_xgb_evals.get_mse()

In [ ]:
test_xgb_evals.get_r2_score()

Training Evaluation

In [ ]:
train_evaluation_data = X_train_preproc.copy()
train_evaluation_data['xgb_preds'] = xgb_model.predict(X_train_preproc)
train_evaluation_data['old_preds'] = 170
train_evaluation_data[response] = training_data[response].reset_index(drop=True)
train_evaluation_data.head()

In [ ]:
train_xgb_evals = XGBRegressorEvaluator(model = xgb_model,
                                       data = train_evaluation_data,
                                       actual_name = response,
                                       expected_name = "xgb_preds",
                                       compare_name='old_preds'
                                       )

In [ ]:
train_xgb_evals.get_mae()

In [ ]:
train_xgb_evals.get_mse()

In [ ]:
train_xgb_evals.get_r2_score()

In [ ]:
train_xgb_evals.plot_shap_summary_plot()

In [ ]:
train_xgb_evals.plot_ave()

In [ ]:
train_xgb_evals.plot_feature_ave('Venue_Docklands')

In [ ]:
train_xgb_evals.plot_feature_ave('Home_Total_Q4_Score_avg2')

Double Lift

In [ ]:
test_xgb_evals.plot_double_lift_chart()

In [ ]:
train_xgb_evals.plot_double_lift_chart()